In [ ]:
import pandas as pd
import re
import os
import numpy as np
from datetime import datetime

## Sheets Usage Stats

Client Billing Descending -- Entire Sheet Not Used 

Fees By Worktype PIE Chart -- Entire Sheet not used  -- No Need to Process This 

Fees Summary by Dept and Fee Earner -- Entire Sheet Not Used

============================================================

Fee Breakdown by Dept and Fee earner -- Yes Except Textbox columns 

Fees Billed -- Yes Except TextBox Columns 

Matters Opened by FE -- Yes Except TextBox Columns 

Matter Sourece of Business Inc Matter Bills - Yes Except TextBox columns 

Payment Received Analysis  - Yes Except TextBox columns (Both)

In [ ]:
skip_rows = {}

skip_rows["Fee Breakdown by Dept and Fee Earner"] = 3
skip_rows["Fees Billed"] = 3
skip_rows["Matters Opened by FE"] = 3
skip_rows["Matter Source of Business inc Matter Bills"] = 0
skip_rows["Payment Received Analysis"] = 3
skip_rows["Total Hours by Fee Earner-With Billings"] = 0

# skip_rows["Fees by Worktype Pie Chart"] = 3
# skip_rows["Fee Summary by Dept and Fee Earner"] = 3
# skip_rows["Client Billing Descending"] = 0

# If a Filename is not in this Dictionary, then it will not be Considered. 

In [ ]:
def remove_cols(df):
    """ 
    This will remove all the columns that contain the word Textbox in them. 
    This Function takes a DataFrame as in input and returns all the columns except TextBox. 

    """
    cols = df.columns
    new_cols = []

    for x in cols:
        txt_chk = re.compile(r'Textbox')
        if txt_chk.search(x)== None:
            new_cols.append(x)
        else: 
            continue
    
    return(new_cols)

In [ ]:
def concat_files(loc, file_name,file_names_array, final_files_loc):
    """
    loc = where these files are 
    file_name = current file name being processed 
    files names array = All the files that need to be concatenated 

    """
    df_final = pd.DataFrame()
    now = datetime.now()
    date = now.strftime("%m-%d-%y") 
    print(f'Processing {file_name}')

    for file in file_names_array:
        dfc_file=pd.read_csv((loc+file), skiprows=get_rows(skip_rows,file))
        dfc_file=dfc_file[remove_cols(dfc_file)]
        
        st_end_dates = get_start_enddate_from_Filename(file)
        dfc_file["From_Date"] = st_end_dates[0]
        print(f'End Date from file is {st_end_dates[1]}')
        dfc_file["To_Date"] = st_end_dates[1]
        df_final = pd.concat([df_final,dfc_file],ignore_index=True)

    # return df_final
    final_file = final_files_loc+file_name+"_"+date+"_.csv"
    df_final.to_csv(final_file,index=False,)
    print(f'Created {final_file}\n')

In [ ]:
def get_start_enddate_from_Filename(fname):
    """
        Accepts a Filename that has fname_date.csv format or fname_datetodate.csv format. 
        It Extracts the From and To Date form the File and Returns 2 Objects 
        These Are Datetime Objects. First is Start Date and Second is End Date. 

        If the Filename has only start date, it will just return the same date for Both Start and End Date. 

    """
    # suffx=fname.split("_")[1]
    # print(f'Suffix is {suffx}')
    # if len(suffx)  > 12:
    #     print("\nIgnoring Monthly Files")
    #     # pattern = re.compile(r'_\d*\w*\d')
        # match = pattern.findall(fname)
        # dt = match[0]
        # dt1 = dt.split("_")[1].split("to")[0]
        # dt2 = dt.split("_")[1].split("to")[1]
        # start_date = pd.to_datetime(dt1,format='%d%m%Y')
        # end_date = pd.to_datetime(dt2,format='%d%m%Y')
    # else:
    #  
    
    pattern = re.compile(r'_\d*')
    match = pattern.findall(fname)
    dt = match[0]
    dt = dt.split("_")[1]
    start_date = pd.to_datetime(dt,format='%d%m%Y')
    end_date = start_date
    
    return start_date,end_date

In [ ]:
def get_rows(dictnry, match):
    """
        Takes a Dictionary and a filename as inputs and Returns how many rows need to be skipped for a filename. 
        Returns the Number of rows to skip, while creating a DataFrame.
    """
    for val in skip_rows.keys():
        if re.match(val,match):
            return (skip_rows[val])

In [ ]:
def get_unique_file_names(files):

    """ 
        This Function will Check a Given Location for all files and find the Unique File Names. 
        It Splits on the "_" as that is the current Naming Convention.
        Also, it only find .csv files
        Returns the Unique File Names.
    """
    
    all_csvs = []
    all_file_names = []

    for file in files:
        if file.endswith('.csv'):
            all_csvs.append(file)

    for file in all_csvs:
        fname = file.split("_")[0]
        all_file_names.append(fname)

    all_file_names =  np.unique(all_file_names)
    return all_file_names

In [ ]:
data_file_loc = "/home/shashankraj/Documents/One_Drv_Anza/test/"
final_xl_files_loc = "/home/shashankraj/Documents/One_Drv_Anza/Final_Files/"
data_files_list = os.listdir(data_file_loc)
data_files_list[:5]

In [ ]:
def final_processing(files_loc,files_list,final_files_loc):
    print("Inside Final Processing")
    unique_files = get_unique_file_names(files_list) ## I need output of os.listdir
    for each_file in unique_files:
        print(f'\nChecking [{each_file}]')
        if each_file == "Matter Source of Business inc Matter Bills (Bill Date)":
            each_file = "Matter Source of Business inc Matter Bills"
        if each_file in skip_rows.keys():
            pattern = re.compile(each_file)
            each_file_arr = [file for file in files_list if len(pattern.findall(file))]
            print(each_file_arr)
            concat_files(files_loc,each_file,each_file_arr,final_files_loc)  
        else: 
            print(f'Skipping Processing of [{each_file}]')
    

In [ ]:
final_processing(data_file_loc, data_files_list,final_xl_files_loc)


#TODO: Find a Way to Break Each File Operation into a Single Thread. Then Use Multithreading or Multiprocessing. 
# TODO: This will be a One Time Operation at a given time. Is it worth the Time Spent?

### Testing

In [ ]:
# loc = "/home/shashankraj/Documents/One_Drv_Anza/test/"
# file_name = "Fee Breakdown by Dept and Fee Earner"
# file_names_array = ['Fee Breakdown by Dept and Fee Earner_11012022.csv', 'Fee Breakdown by Dept and Fee Earner_20122021.csv', 
# 'Fee Breakdown by Dept and Fee Earner_18112021.csv']

# final  = concat_files(loc,file_name,file_names_array)

In [ ]:
# file_loc = "/home/shashankraj/Documents/One_Drv_Anza/test/"

# c_file = os.listdir(file_loc)
# c_file = c_file[5]
# c_file

In [ ]:
# df_final = pd.DataFrame()
# dfc_file = pd.read_csv((file_loc+c_file), skiprows=get_rows(skip_rows,c_file))
# dfc_file

In [ ]:
# dfc_file = dfc_file[remove_cols(dfc_file)]
# dfc_file

In [ ]:
# st_end_dates = get_start_enddate_from_Filename(c_file)
# dfc_file["From_Date"] = st_end_dates[0]
# print(f'Start Date from file is {st_end_dates[0]}')
# dfc_file["To_Date"] = st_end_dates[1]
# dfc_file

In [ ]:
# location = "/home/shashankraj/Documents/One_Drv_Anza/test"
# curr_file = get_unique_file_names(os.listdir(location))[5]
# curr_file

# pattern = re.compile(r'Matter Source of Business inc Matter Bills')
# x = [file for file in os.listdir(location) if len(pattern.findall(file))]

# print(x)

# print("*"*50)

# print(len(x))

# concat_files(location,curr_file,x)